Intro bla bla

In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import joblib
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from google.cloud import storage
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import numpy as np


In [3]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b8/9d/1ce80cee7c5ef60f2fcc7e9fa97f29f7a8de3dc5a08922b3b2f1e9106481/lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl.metadata
  Using cached lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl.metadata (19 kB)
Using cached lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl (3.1 MB)


In [75]:
# define the common path
# change this to your local path
path = 'gs://sleep-nov2023/'

In [76]:
# Read data
X_train1 = pd.read_parquet(path + 'data/full_train_and_split/X_train_lids.parquet')
y_train1 = pd.read_parquet(path + 'data/full_train_and_split/y_train_lids_match.parquet')
#X_test = pd.read_parquet(path + 'full_train_and_split/X_test.parquet')
#y_test = pd.read_parquet(path + 'full_train_and_split/y_test.parquet')

# Extract 'hour' and 'minute_of_day' from the timestamp
#X_train['hour'] = pd.to_datetime(X_train['timestamp']).dt.hour
#X_train['minute_of_day'] = pd.to_datetime(X_train['timestamp']).dt.hour * 60 + pd.to_datetime(X_train['timestamp']).dt.minute

#X_test['hour'] = pd.to_datetime(X_test['timestamp']).dt.hour
#X_test['minute_of_day'] = pd.to_datetime(X_test['timestamp']).dt.hour * 60 + pd.to_datetime(X_test['timestamp']).dt.minute


In [77]:
y_train1.shape

(7778985, 9)

In [78]:
y_train1.head()

,series_id,timestamp,step,night,event,year,month,day,hour
0,038441c925bb,2018-08-14 15:30:00,0.0,NaN,None,NaN,NaN,NaN,NaN
1,038441c925bb,2018-08-14 15:31:00,12.0,NaN,None,NaN,NaN,NaN,NaN
2,038441c925bb,2018-08-14 15:32:00,24.0,NaN,None,NaN,NaN,NaN,NaN
3,038441c925bb,2018-08-14 15:33:00,36.0,NaN,None,NaN,NaN,NaN,NaN
4,038441c925bb,2018-08-14 15:34:00,48.0,NaN,None,NaN,NaN,NaN,NaN


In [79]:
y_train1['event'].fillna('no_event', inplace=True)

In [80]:
y_train1['event'].value_counts()

event
no_event    7771848
wakeup         3570
onset          3567
Name: count, dtype: int64

In [81]:
y_train1['event'].value_counts(normalize=True)

event
no_event    0.999083
wakeup      0.000459
onset       0.000459
Name: proportion, dtype: float64

In [11]:
label_encoder = LabelEncoder()
y_train1['event_encoded'] = label_encoder.fit_transform(y_train1['event'])

In [ ]:
y_train1['event_encoded'].value_counts(normalize=True)

In [12]:
X_train1.shape

(7778985, 6)

In [13]:
X_train1['hour'] = pd.to_datetime(X_train1['timestamp']).dt.hour
X_train1['minute_of_day'] = pd.to_datetime(X_train1['timestamp']).dt.hour * 60 + pd.to_datetime(X_train1['timestamp']).dt.minute

# Drop 'series_id' and 'timestamp'
X_train1.drop(['series_id', 'timestamp', 'step'], axis=1, inplace=True)

In [14]:
X_train1.head()

,anglez,enmo,lids,hour,minute_of_day
0,3.084700,0.0229,0.0,15,930
1,68.460503,0.0395,0.0,15,931
2,-79.998703,0.0691,0.0,15,932
3,-80.008202,0.0140,0.0,15,933
4,-80.025299,0.0141,0.0,15,934


In [15]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1['event_encoded'], test_size=0.2, random_state=42)


In [16]:
# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', KNNImputer()),
            ('scaler', StandardScaler())
        ]), ['anglez', 'enmo', 'lids', 'hour', 'minute_of_day'])
    ]
)

# Logistic Regression

In [62]:
# Create the logistic regression model with class weights
logreg_model = LogisticRegression(class_weight='balanced', max_iter=1000)

In [63]:
# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logreg_model)
])

In [64]:
# Train the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['anglez', 'enmo', 'lids',
                                                   'hour',
                                                   'minute_of_day'])])),
                ('classifier',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [65]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

In [66]:
classification_report_result = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report_result)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.35      0.52   1554402
           1       0.00      0.78      0.00       734
           2       0.00      0.90      0.00       661

    accuracy                           0.35   1555797
   macro avg       0.33      0.68      0.17   1555797
weighted avg       1.00      0.35      0.52   1555797



# Hyperparameters

In [67]:
# Define the parameter distributions for logistic regression
logreg_param_dist = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear'],
    'classifier__class_weight': ['balanced']
}

In [68]:
# Create the logistic regression model without specifying hyperparameters
logreg_model = LogisticRegression()


In [69]:
# Define a function to convert input to DataFrame if it's a NumPy array
def to_dataframe(X):
    if isinstance(X, np.ndarray):
        return pd.DataFrame(X)
    return X

In [70]:
# Define the preprocessor with 'remainder' set to 'passthrough'
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('to_dataframe', FunctionTransformer(to_dataframe, validate=False)),
            ('imputer', KNNImputer()),
            ('scaler', StandardScaler())
        ]), ['anglez', 'enmo', 'lids', 'hour', 'minute_of_day'])
    ],
    remainder='passthrough'
)


In [71]:
# Create the pipeline with the preprocessor and logistic regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logreg_model)
])

In [72]:
# Create the randomized search with cross-validation
logreg_random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=logreg_param_dist,
    n_iter=10,  # Adjust the number of iterations as needed
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

In [73]:
# Train the randomized search on the training data
logreg_random_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Get the best parameters and best estimator from the randomized search
best_params_logreg = logreg_random_search.best_params_
best_estimator_logreg = logreg_random_search.best_estimator_

In [ ]:
# Make predictions on the test set using the best estimator
y_pred_best_logreg = best_estimator_logreg.predict(X_test)

In [ ]:
# Display the best parameters and classification report
print("Best Parameters:", best_params_logreg)
print("\nClassification Report (Best Estimator):")
print(classification_report(y_test, y_pred_best))

# Saving to Google Cloud Storage

In [ ]:
# Save the model file to a local directory
joblib.dump(best_estimator_logreg, 'tuned_logreg_model.joblib')

In [ ]:
# Specify your Google Cloud Storage bucket and file path
bucket_name = 'sleep-nov2023'
destination_blob_name = 'Model/computed_models/tuned_logreg_model.joblib'

In [ ]:
# Create a storage client
storage_client = storage.Client()

In [ ]:
# Create a bucket object
bucket = storage_client.bucket(bucket_name)

In [ ]:
# Specify the local file path
local_file_path = 'tuned_logreg_model.joblib'

In [ ]:
# Upload the model file to Google Cloud Storage
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(local_file_path)

# XGB Classifier

In [42]:
# Calculate class weights
class_weights = len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))


In [43]:
# Create the XGBoost model with class weights
xgb_model = XGBClassifier(
    learning_rate=0.1,
    n_estimators=100,
    scale_pos_weight=class_weights[1]
)

In [44]:
# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_model)
])

In [45]:
# Train the pipeline on the training data
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:21:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['anglez', 'enmo', 'lids',
                                                   'hour',
                                                   'minute_of_day'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [46]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

In [47]:
# Display classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1554402
           1       0.00      0.00      0.00       734
           2       0.00      0.00      0.00       661

    accuracy                           1.00   1555797
   macro avg       0.33      0.33      0.33   1555797
weighted avg       1.00      1.00      1.00   1555797



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hyperparameter

In [48]:
# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'classifier__learning_rate': uniform(0.01, 0.3),
    'classifier__n_estimators': randint(50, 200),
    'classifier__max_depth': randint(3, 10),
    'classifier__min_child_weight': randint(1, 10),
    'classifier__subsample': uniform(0.6, 0.4),
    'classifier__colsample_bytree': uniform(0.6, 0.4),
    'classifier__gamma': uniform(0, 0.5),
}


In [49]:
# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    random_state=42
)

In [50]:
# Train the randomized search on the training data
random_search.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:22:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:22:48] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:22:48] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:22:49] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:22:49] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weigh

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                KNNImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['anglez',
                                                                                'enmo',
                                                                                'lids',
                                                                                'hour',
                                                                                'minute_of_day'])])),
                                             ('classifier',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsa...
                                        'classifier__min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd89bd6f040>,
                                        'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fd89bd6ef50>,
                                        'classifier__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd89bd2e110>},
                   random_state=42, scoring='accuracy')

In [51]:
# Get the best parameters from the random search
best_params_xgb = random_search.best_params_

In [52]:
# Create a new pipeline with the best parameters and preprocessing steps
best_estimator_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**best_params_xgb))
])

In [53]:
# Fit the pipeline on the training data
best_estimator_xgb.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:34:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "classifier__colsample_bytree", "classifier__gamma", "classifier__learning_rate", "classifier__max_depth", "classifier__min_child_weight", "classifier__n_estimators", "classifier__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['anglez', 'enmo', 'lids',
                                                   'hour',
                                                   'minute_of_day'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               classifier__colsample_bytree=0.749816047538945,
                               classifier__gamma=0.4753571532049581,...
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, ...))])

In [54]:
# Make predictions on the test set using the best estimator
y_pred_best_xgb = best_estimator_xgb.predict(X_test)

In [55]:
# Display the best parameters and classification report for XGBoost
print("Best Parameters (XGBoost):", best_params_xgb)
print("\nClassification Report (Best Estimator - XGBoost):")
print(classification_report(y_test, y_pred_best_xgb))

Best Parameters (XGBoost): {'classifier__colsample_bytree': 0.749816047538945, 'classifier__gamma': 0.4753571532049581, 'classifier__learning_rate': 0.22959818254342154, 'classifier__max_depth': 7, 'classifier__min_child_weight': 5, 'classifier__n_estimators': 152, 'classifier__subsample': 0.7783331011414365}

Classification Report (Best Estimator - XGBoost):


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1554402
           1       0.00      0.00      0.00       734
           2       0.00      0.00      0.00       661

    accuracy                           1.00   1555797
   macro avg       0.33      0.33      0.33   1555797
weighted avg       1.00      1.00      1.00   1555797



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Saving to Google Cloud Storage

In [56]:
# Save the model file to a local directory
joblib.dump(best_estimator_xgb, 'tuned_xgb_model.joblib')

['tuned_xgb_model.joblib']

In [57]:
# Specify your Google Cloud Storage bucket and file path
bucket_name = 'sleep-nov2023'
destination_blob_name = 'Model/computed_models/tuned_xgb_model.joblib'

In [58]:
# Create a storage client
storage_client = storage.Client()

In [59]:
# Create a bucket object
bucket = storage_client.bucket(bucket_name)

In [60]:
# Specify the local file path
local_file_path = 'tuned_xgb_model.joblib'

In [61]:
# Upload the model file to Google Cloud Storage
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(local_file_path)

# LGBM Classifier

In [95]:
# Calculate class weights
class_weights = {0: len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))[0],
                 1: len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))[1],
                 2: len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))[2]}

In [96]:
# Define the LGBM model with class weights
lgbm_model = LGBMClassifier(
    objective='multiclass',
    num_class=len(np.unique(y_train)),
    learning_rate=0.1,
    n_estimators=100,
    class_weight=class_weights,  # Pass the dictionary of class weights
    metric='multi_logloss'
)


In [97]:
# Create the pipeline for LGBM
pipeline_lgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgbm_model)
])


In [98]:
# Train the pipeline on the training data
pipeline_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050707 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1045
[LightGBM] [Info] Number of data points in the train set: 6223188, number of used features: 5
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['anglez', 'enmo', 'lids',
                                                   'hour',
                                                   'minute_of_day'])])),
                ('classifier',
                 LGBMClassifier(class_weight={0: 0.3336411767790183,
                                              1: 732.2259089304624,
                                              2: 713.095909247164},
                                metric='multi_logloss', num_class=3,
                                objective='multiclass'))])

In [99]:
# Make predictions on the test set
y_pred_lgbm = pipeline_lgbm.predict(X_test)

In [100]:
# Display classification report
class_report_lgbm = classification_report(y_test, y_pred_lgbm)
print('Classification Report (LGBM):')
print(class_report_lgbm)

Classification Report (LGBM):
              precision    recall  f1-score   support

           0       1.00      0.78      0.88   1554402
           1       0.00      0.88      0.01       734
           2       0.00      0.92      0.01       661

    accuracy                           0.78   1555797
   macro avg       0.34      0.86      0.30   1555797
weighted avg       1.00      0.78      0.87   1555797



# Hyperparameter

In [126]:
# Define the parameter grid for LGBM
lgbm_param_grid = {
    'classifier__num_leaves': [20, 30, 40, 50],
    'classifier__max_depth': [5, 10, 15, 20],
    'classifier__min_child_samples': [10, 20, 30],
    'classifier__subsample': [0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.8, 0.9, 1.0]
}

In [129]:
# Create the randomized grid search for LGBM
lgbm_random_search = RandomizedSearchCV(
    pipeline_lgbm,
    param_distributions=lgbm_param_grid,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

In [130]:
# Train the randomized search on the training data
lgbm_random_search.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.926970 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1045
[LightGBM] [Info] Number of data points in the train set: 4978551, number of used features: 5
[LightGBM] [Info] Start training from score -1.098672
[LightGBM] [Info] Start training from score -1.098407
[LightGBM] [Info] Start training from score -1.098758
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                KNNImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['anglez',
                                                                                'enmo',
                                                                                'lids',
                                                                                'hour',
                                                                                'minute_of_day'])])),
                                             ('classifier',
                                              LGBMClassifier(class_weight={0: 0.3336411767790183,
                                                                           1: 732.2259089304624,
                                                                           2: 713.095909247164},
                                                             metric='multi_logloss',
                                                             num_class=3,
                                                             objective='multiclass'))]),
                   n_jobs=-1,
                   param_distributions={'classifier__colsample_bytree': [0.8,
                                                                         0.9,
                                                                         1.0],
                                        'classifier__max_depth': [5, 10, 15,
                                                                  20],
                                        'classifier__min_child_samples': [10,
                                                                          20,
                                                                          30],
                                        'classifier__num_leaves': [20, 30, 40,
                                                                   50],
                                        'classifier__subsample': [0.8, 0.9,
                                                                  1.0]},
                   random_state=42, scoring='accuracy')

In [131]:
# Get the best parameters and best estimator from the randomized search
best_params_lgbm = lgbm_random_search.best_params_
best_estimator_lgbm = lgbm_random_search.best_estimator_

In [132]:
# Make predictions on the test set using the best estimator
y_pred_best_lgbm = best_estimator_lgbm.predict(X_test)

In [133]:
# Display the best parameters and classification report for LGBM
print("Best Parameters (LGBM):", best_params_lgbm)
print("\nClassification Report (Best Estimator - LGBM):")
print(classification_report(y_test, y_pred_best_lgbm))

Best Parameters (LGBM): {'classifier__subsample': 1.0, 'classifier__num_leaves': 50, 'classifier__min_child_samples': 20, 'classifier__max_depth': 20, 'classifier__colsample_bytree': 0.9}

Classification Report (Best Estimator - LGBM):
              precision    recall  f1-score   support

           0       1.00      0.80      0.89   1554402
           1       0.00      0.85      0.01       734
           2       0.00      0.89      0.01       661

    accuracy                           0.80   1555797
   macro avg       0.34      0.85      0.30   1555797
weighted avg       1.00      0.80      0.89   1555797



# Saving to Google Cloud Storage

In [134]:
# Save the model file to a local directory
joblib.dump(best_estimator_lgbm, 'tuned_lgbm_model.joblib')

['tuned_lgbm_model.joblib']

In [135]:
# Specify your Google Cloud Storage bucket and file path
bucket_name = 'sleep-nov2023'
destination_blob_name = 'Model/computed_models/tuned_lgbm_model.joblib'

In [136]:
# Create a storage client
storage_client = storage.Client()

In [137]:
# Create a bucket object
bucket = storage_client.bucket(bucket_name)

In [138]:
# Specify the local file path
local_file_path = 'tuned_lgbm_model.joblib'

In [139]:
# Upload the model file to Google Cloud Storage
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(local_file_path)

# KNN

In [101]:
# Define the KNN model with class weights
knn_model = KNeighborsClassifier(
    n_neighbors=5,  # Adjust the number of neighbors as needed
    weights='uniform',  # or 'distance' based on preference
)

In [106]:
# Define the preprocessor for KNN
numeric_features = ['anglez', 'enmo', 'lids', 'hour', 'minute_of_day']
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer())
])

preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

In [107]:
# Create the pipeline for KNN
pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor_knn),
    ('classifier', knn_model)
])

In [108]:
# Train the pipeline on the training data
pipeline_knn.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer())]),
                                                  ['anglez', 'enmo', 'lids',
                                                   'hour',
                                                   'minute_of_day'])])),
                ('classifier', KNeighborsClassifier())])

In [109]:
# Make predictions on the test set
y_pred_knn = pipeline_knn.predict(X_test)

In [110]:
# Display classification report
class_report_knn = classification_report(y_test, y_pred_knn)
print('Classification Report (KNN):')
print(class_report_knn)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report (KNN):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1554402
           1       0.00      0.00      0.00       734
           2       0.00      0.00      0.00       661

    accuracy                           1.00   1555797
   macro avg       0.33      0.33      0.33   1555797
weighted avg       1.00      1.00      1.00   1555797



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hyperparameter

In [114]:
# Define the parameter grid for KNN
knn_param_grid = {
    'classifier__n_neighbors': [3, 5, 7, 10],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

In [115]:
# Create the randomized grid search for KNN
knn_random_search = RandomizedSearchCV(
    pipeline_knn,
    param_distributions=knn_param_grid,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

In [116]:
# Train the randomized search on the training data
knn_random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                KNNImputer())]),
                                                                               ['anglez',
                                                                                'enmo',
                                                                                'lids',
                                                                                'hour',
                                                                                'minute_of_day'])])),
                                             ('classifier',
                                              KNeighborsClassifier())]),
                   n_jobs=-1,
                   param_distributions={'classifier__n_neighbors': [3, 5, 7,
                                                                    10],
                                        'classifier__p': [1, 2],
                                        'classifier__weights': ['uniform',
                                                                'distance']},
                   random_state=42, scoring='accuracy')

In [117]:
# Get the best parameters and best estimator from the randomized search
best_params_knn = knn_random_search.best_params_
best_estimator_knn = knn_random_search.best_estimator_

In [118]:
# Make predictions on the test set using the best estimator
y_pred_best_knn = best_estimator_knn.predict(X_test)

In [119]:
# Display the best parameters and classification report for KNN
print("Best Parameters (KNN):", best_params_knn)
print("\nClassification Report (Best Estimator - KNN):")
print(classification_report(y_test, y_pred_best_knn))

Best Parameters (KNN): {'classifier__weights': 'uniform', 'classifier__p': 2, 'classifier__n_neighbors': 10}

Classification Report (Best Estimator - KNN):


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1554402
           1       0.00      0.00      0.00       734
           2       0.00      0.00      0.00       661

    accuracy                           1.00   1555797
   macro avg       0.33      0.33      0.33   1555797
weighted avg       1.00      1.00      1.00   1555797



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Saving to Google Cloud Storage

In [120]:
# Save the model file to a local directory
joblib.dump(best_estimator_knn, 'tuned_knn_model.joblib')

['tuned_knn_model.joblib']

In [121]:
# Specify your Google Cloud Storage bucket and file path
bucket_name = 'sleep-nov2023'
destination_blob_name = 'Model/computed_models/tuned_knn_model.joblib'


In [122]:
# Create a storage client
storage_client = storage.Client()

In [123]:
# Create a bucket object
bucket = storage_client.bucket(bucket_name)

In [124]:
# Specify the local file path
local_file_path = 'tuned_knn_model.joblib'

In [125]:
# Upload the model file to Google Cloud Storage
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(local_file_path)